In [1]:
%run "Setup_DesignStart.ipynb"

Serial baud rate = 38400


In [10]:
target.fpga_read(target.REG_SYNCHRONIZED, 1)

Traceback (most recent call last):
  File "c:\users\jp\github\cw_develop\software\chipwhisperer\hardware\naeusb\naeusb.py", line 287, in txrx
    response = self.cmdReadMem(addr, dlen)
  File "c:\users\jp\github\cw_develop\software\chipwhisperer\hardware\naeusb\naeusb.py", line 465, in cmdReadMem
    self.sendCtrl(cmd, data=pload)
  File "c:\users\jp\github\cw_develop\software\chipwhisperer\hardware\naeusb\naeusb.py", line 440, in sendCtrl
    return self.usbdev().ctrl_transfer(0x41, cmd, value, 0, data, timeout=self._timeout)
  File "C:\ProgramData\Anaconda3\lib\site-packages\usb\core.py", line 1043, in ctrl_transfer
    self.__get_timeout(timeout))
  File "C:\ProgramData\Anaconda3\lib\site-packages\usb\backend\libusb0.py", line 593, in ctrl_transfer
    timeout
  File "C:\ProgramData\Anaconda3\lib\site-packages\usb\backend\libusb0.py", line 431, in _check
    raise USBError(errmsg, ret)
usb.core.USBError: [Errno None] b'libusb0-dll:err [control_msg] sending control message failed, wi

USBError: [Errno None] None

In [ ]:
target.fpga_write(target.REG_ARM, [0])

In [ ]:
target.REG_ARM

In [3]:
target.get_target_name()

'ARMTRACE'

In [9]:
target.get_fw_buildtime()

DesignStart Cortex M3, compiled Jun 29 2020, 15:06:00


In [5]:
target.get_fpga_buildtime()

'FPGA build time: 1/17/2004, 0:16'

In [ ]:
target.REG_NAME

In [6]:
target.get_reg('DWT_COMP0')

'000003E6'

In [ ]:
# PC value that will generate a trace packet:
target.set_reg('DWT_COMP0', '000004aa')

In [ ]:
# match on any PC trace packet:
target.set_pattern_match(0, [5, 8, 32, 0, 0, 0, 0, 0], [255, 255, 255, 0, 0, 0, 0, 0])

# enable matching rule:
target.fpga_write(target.REG_PATTERN_ENABLE, [1])

In [8]:
# have target encrypt:
target.simpleserial_write('p', bytearray(16))

In [ ]:
# report trace match address:
print(hex(target.get_trace_match_address()))

In [ ]:
# do a bunch of encryptions; each causes a PC match; check that all are detected:
start_count = target.fpga_read(target.REG_TRACE_COUNT0, 1)[0]
runs = 100
for i in range(runs):
    target.simpleserial_write('p', bytearray(16))
    time.sleep(0.05)
matches = target.fpga_read(target.REG_TRACE_COUNT0, 1)[0] - start_count
assert matches == runs